<a href="https://colab.research.google.com/github/charlietena200/machine_programs/blob/main/Reconcer_caras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=e050503c48c5c1ca853e42e237eec6f93352be315316fb2b2c701f759a413c23
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [3]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js  #ejecutar javascript
from base64 import b64decode, b64encode #decodificar imágenes
import cv2 #opencv
import numpy as np
import PIL #manipular imágenes
import io #leer escribir datos
import html
import time
import face_recognition  #biblioteca de reconocimiento facial

In [ ]:
# function
def js_to_image(js_reply):   #convertir imagen base 64 a BGR de OpenCV
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decodificar imagen
  image_bytes = b64decode(js_reply.split(',')[1])
  # convertir bytes en arreglo numpy
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # usar el arreglo a Opencv
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# arreglo numpy de rectángulo delimitador (bounding box) en una cadena de bytes base64 para superponerla
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # arreglo a PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # bbox a png
  bbox_PIL.save(iobuf, format='png')
  # string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

In [ ]:
carlos_image = face_recognition.load_image_file("/content/sample_data/foto mia.jpeg") #cargar imágenes
carlos_face_encoding = face_recognition.face_encodings(carlos_image)[0] # imagen de entrada y devuelve un arreglo de numpy que representa la codificación facial

mau_image = face_recognition.load_image_file("/content/sample_data/mawiii.jpg")
mau_face_encoding = face_recognition.face_encodings(mau_image)[0]

profesor_image = face_recognition.load_image_file("/content/sample_data/profesor.jpg")
profesor_face_encoding = face_recognition.face_encodings(profesor_image)[0]

known_face_encodings = [
    carlos_face_encoding,
    mau_face_encoding,
    profesor_face_encoding
]
known_face_names = [
    "Carlos Gallegos",
    "Mauricio",
    "Profesor Gustavo"
]


In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  cont=0
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  #
  #
  #
  face_locations = face_recognition.face_locations(img)
  face_encodings = face_recognition.face_encodings(img, face_locations)
  #
  #
  for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
    name = "Unknown"
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
      name = known_face_names[best_match_index]
      cont=1
    cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
    cv2.rectangle(img, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(img, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
  # draw face bounding box on image
  #for (x,y,w,h) in faces:
      #img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # save image
  cv2.imwrite(filename, img)

  return filename,cont

In [ ]:
filename,llave = take_photo('photo.jpg')
print('Su foto ha sido guardada en {}'.format(filename))
if llave==1:
  print("Bienvenido Profesor, tome su llave")
else:
  print("Lo siento, no está autorizado")

<IPython.core.display.Javascript object>

Su foto ha sido guardada en photo.jpg
Bienvenido Profesor, tome su llave


In [1]:
from PIL import Image

# Carga la imagen
image = Image.open("photo.jpg")

# Muestra la imagen
image.show()





FileNotFoundError: [Errno 2] No such file or directory: 'photo.jpg'